In [53]:
import pyvista as pv
# import topogenesis  as tp
import numpy as np
from honeybee_plus.hbsurface import HBSurface
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid
import os
import ladybug as lb
import pandas as pd

cwd = os.getcwd()
src = os.path.dirname(cwd)

In [54]:
# srf_dict = {
#     'name': 'floor', 
#     'vertices': [[(0, 0, 3.2), (-4.242640687119285, 4.242640687119286, 3.2), (-1.2727922061357848, 7.212489168102785, 3.2), (2.9698484809835, 2.9698484809834995, 3.2)]], 
#     'surface_material': {
#         'modifier': 'void', 
#         'type': 'plastic', 
#         'name': 'generic_floor', 
#         'r_reflectance': 0.2, 
#         'g_reflectance': 0.2, 
#         'b_reflectance': 0.2, 
#         'specularity': 0.0, 
#         'roughness': 0.0}, 
#     'surface_type': 2.0
# }
def mesh_to_hbsurface(mesh, s_type):
    hb_surfaces = []
    face_list = list(mesh.faces)
    e = 0
    while e < len(face_list):
        v_count = face_list[e]
        vertices = []
        for v in range(v_count):
            e += 1
            vertices.append(list(mesh.points[face_list[e]]))
        srf_dict = {
            "name": "",
            "vertices": vertices, # [[(x, y, z), (x1, y1, z1), (x2, y2, z2)]],
            "surface_material": {
                'modifier': 'void', 
                'type': 'plastic', 
                'name': 'generic_floor', 
                'r_reflectance': 0.2, 
                'g_reflectance': 0.2, 
                'b_reflectance': 0.2, 
                'specularity': 0.0, 
                'roughness': 0.0},
            "surface_type": s_type  # 0: wall, 5: window 
            # TODO: look for the lables of ceiling and floor
            }
        hbsrf = HBSurface.from_json(srf_dict)
        hb_surfaces.append(hbsrf)
        e += 1
    return hb_surfaces

In [55]:
# import all external geometry with pyvista
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = pv.read(wal_filepath)
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = pv.read(fen_filepath)

# construct honeybee surfaces from the mesh
wal_hb_surf = mesh_to_hbsurface(wal_mesh, 0)
fen_hb_surf = mesh_to_hbsurface(fen_mesh, 5)
hb_surfaces = wal_hb_surf + fen_hb_surf
# TODO: look into tagging system for out_view to specify the "external environment"

# print(hb_surfaces)
# print(type(mesh))
# print(mesh.points)
# print(mesh.faces)

In [56]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# initiate sunpath
sp = lb.sunpath.Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
for i in range(8760):
    # compute the sun object
    sun = sp.calculate_sun_from_hoy(i)
    # extract the sun vector
    sun_vector = sun.sun_vector.to_array()
    # apparantly, if the Z component of sun vector is positive, it is under the horizon 
    if sun_vector[2] < 0.0:
        hoys.append(i)
        sun_vectors.append(sun_vector)

# put the recipe together
rp = SolarAccessGridBased(sun_vectors, hoys, [analysis_grid], hb_objects=hb_surfaces)

Found 78 opaque surfaces.
Found 0 fenestration surfaces.
Found 0 window-groups.


In [57]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='external_room_sun')

Writing recipe contents to: ./external_room_sun/solaraccess


In [58]:
# print the radiance commands
for com in rp._commands:
    print(com)
# run the simulation
rp.run(batch_file, debug=False)

# https://www.radiance-online.org/learning/documentation/manual-pages/pdfs/rmtxop.pdf
# Eleonora: rmtxop multiplies the daylight coefficient matrices with the sky vectors. This is necessary for getting illuminance from daylight coefficients computed w.r.t. rgb components. 
# Sun Access of points, and how to interpret them https://floyd.lbl.gov/radiance/refer/usman2.pdf

#!/usr/bin/env bash



cd ./external_room_sun/solaraccess

/usr/local/radiance/bin/oconv -f scene/opaque/external_room_sun..opq.mat scene/opaque/external_room_sun..opq.rad scene/glazing/external_room_sun..glz.mat scene/glazing/external_room_sun..glz.rad sky/analemma.rad > external_room_sun.oct
/usr/local/radiance/bin/rcontrib -aa 0.0 -ab 0 -ad 512 -ar 16 -as 128 -dc 1.0 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.0 -I -lr 4 -lw 0.05 -M ./sky/analemma.mod -ss 0.0 -st 0.85 -y 4 external_room_sun.oct < external_room_sun.pts > result/external_room_sun.dc
/usr/local/radiance/bin/rmtxop -c 47.4 119.9 11.6 -fa result/external_room_sun.dc > result/external_room_sun.ill


True

In [66]:
# results - in this case it will be an analysis grid
result = rp.results()[0]
dfs = []

# print(result.result_files)
for i, ap in enumerate(result._analysis_points):
    data_dict = ap._values[0][0]
    df = pd.DataFrame(data_dict)
    dft = df.T
    dft.columns = [i, i+1]
    dfs.append(dft[i])

df = pd.concat(dfs, axis=1)
print(df)
print(sum(df[0])/4460.0)
print(sum(df[1])/4460.0)
print(sum(df[2])/4460.0)
print(sum(df[3])/4460.0)
# df = pd.DataFrame({
#   'IX': vol_3d_ind_flat[:, 0],
#   'IY': vol_3d_ind_flat[:, 1],
#   'IZ': vol_3d_ind_flat[:, 2],
#   'value': vol_flat,
#})
# print result with legend
# leg = ["Daylight autonomy", "Continious daylight autonomy", "Useful daylight illuminance", "Less than UDI", "More than UDI"]
# for l, d in zip(leg, result.annual_metrics(da_threshhold=1)):
#    print(l, d)

# extract the points of interest from the recipe
result_points = [[p[0], p[1], p[2]] for p in result.points]

Unloading the current values from the analysis grids.
          0    1    2    3
540     0.0  1.0  0.0  1.0
600     0.0  1.0  0.0  1.0
660     0.0  1.0  0.0  1.0
720     0.0  1.0  0.0  1.0
780     0.0  1.0  0.0  1.0
...     ...  ...  ...  ...
524820  0.0  1.0  0.0  1.0
524880  0.0  1.0  0.0  1.0
524940  0.0  1.0  0.0  1.0
525000  0.0  1.0  0.0  1.0
525060  0.0  1.0  0.0  1.0

[4460 rows x 4 columns]
0.0
0.707847533632287
0.4744394618834081
1.0


In [65]:
# Plot using the ITKplotter
pl = pv.PlotterITK()
# add the external wall mesh
pl.add_mesh(wal_mesh, smooth_shading=True)
# add the external fens mesh, transparrent blue
pl.add_mesh(fen_mesh, smooth_shading=True, opacity=0.3, color='#c1e2ff')
# add points of interest : blue
pl.add_points(pv.PolyData(result_points), color='#2499ff')
# add sun points : orange
pl.add_points(pv.PolyData(np.array(sun_vectors) * -20), color='#ffa500')
# show the plot
pl.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…